In [1]:
import polars as pl
import numpy as np
from tqdm import tqdm
from scipy.sparse import csr_matrix
import implicit

RANDOM_STATE = 42
N_PREDICTIONS = 100

C:\Users\lozhn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pl.read_parquet("C:\labs\hh_hack\hh_recsys_train_hh.pq")
test = pl.read_parquet("C:\labs\hh_hack\hh_recsys_test_hh.pq")
vac = pl.read_parquet("C:\labs\hh_hack\hh_recsys_vacancies.pq")

In [3]:
new_df = pl.DataFrame()
for i in tqdm(range(1, 70)):
    new_df = pl.concat([new_df, train[(i-1)*50_000:i*50_000].explode(columns=["vacancy_id", "action_type", "action_dt"]).sort("action_dt")])
new_df.head()

100%|██████████| 69/69 [00:27<00:00,  2.47it/s]


user_id,session_id,vacancy_id,action_type,action_dt
str,str,str,i64,datetime[ns]
"""u_370846""","""s_24698241""","""v_697571""",1,2023-11-01 00:00:00.919
"""u_461521""","""s_7559925""","""v_2514797""",2,2023-11-01 00:00:06.973
"""u_332204""","""s_6570164""","""v_742810""",2,2023-11-01 00:00:12.594
"""u_229153""","""s_23936793""","""v_1411424""",2,2023-11-01 00:00:13.518
"""u_1128173""","""s_14266530""","""v_248154""",2,2023-11-01 00:00:16.114


In [4]:
train_df = new_df

In [10]:
unique_users = train_df["user_id"].unique().to_list()
unique_vacancies = train_df["vacancy_id"].unique().to_list()

user2idx = {user_id: idx for idx, user_id in enumerate(unique_users)}
vac2idx = {vac_id: idx for idx, vac_id in enumerate(unique_vacancies)}

In [11]:
idx2vac = {idx: vac_id for vac_id, idx in vac2idx.items()}
action_weights = {
    1: 4.0,
    2: 1.0,
    3: 2.0
}

In [6]:
pairs = train_df.select(["user_id", "vacancy_id", "action_type"])

In [13]:
users = pairs["user_id"].replace(user2idx).to_numpy()
vacancies = pairs['vacancy_id'].replace(vac2idx).to_numpy()
preferences = pairs['action_type'].replace(action_weights).to_numpy()

SeriesView([4., 1., 1., ..., 1., 1., 1.])

In [8]:
uv_mat = csr_matrix((preferences, (users, vacancies)))
uv_mat

user_id,vacancy_id,action_type
str,str,i64
"""u_370846""","""v_697571""",1
"""u_461521""","""v_2514797""",2
"""u_332204""","""v_742810""",2
"""u_229153""","""v_1411424""",2
"""u_1128173""","""v_248154""",2
"""u_229153""","""v_1411424""",1
"""u_370846""","""v_1808817""",2
"""u_797043""","""v_664053""",1
"""u_797043""","""v_664053""",2
